**Задание 1**
Скачайте с сайта https://grouplens.org/datasets/movielens/ датасет любого размера. Определите какому фильму было выставлено больше всего оценок 5.0.

In [128]:
import pandas as pd
ratings = pd.read_csv('ratings.csv')
movies= pd.read_csv('movies.csv')
ratings_max=ratings.query('rating == {}'.format(5))
ratings_max.head()
ratings_max['movieId'].value_counts().head()

318     153
296     123
356     116
2571    109
260     104
Name: movieId, dtype: int64

**Задание 2**
По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

In [76]:
import pandas as pd
data = pd.read_csv('power.csv')
data.head()
data=data[ ((data['country']=='Estonia') | (data['country']=='Latvian') | (data['country']=='Lithuania')) & ( (data['category']==4) | (data['category']==12) | (data['category']==21) ) & (2005<=data['year']) & (data['year']<=2010) & ( data['quantity']>=0 ) ]
data['quantity'].count()                                                                         

316

**Задание 3**
Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):
https://fortrader.org/quotes
https://www.finanz.ru/valyuty/v-realnom-vremeni


In [1]:
import pandas as pd
page_url='https://www.finanz.ru/valyuty/czk-rub'
data=pd.read_html(page_url, attrs = {'class': 'news_table'}, encoding='utf-8')
data[0]

,Дата,Закрытие,Открытие,Максимум,Минимум
0,06.03.21,33662,33697,33697,33662
1,05.03.21,33654,34047,34063,33561
2,04.03.21,34070,34096,34149,33665
3,03.03.21,34089,34061,34161,33917
4,02.03.21,34055,34245,34329,33855
5,01.03.21,34282,34370,34429,34073
6,28.02.21,34462,34404,34498,34397
7,27.02.21,34404,34404,34404,34404
8,26.02.21,34407,34680,34711,34264
9,25.02.21,34674,34358,34757,34308


**Задание 4**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

In [33]:
import pandas as pd
ratings = pd.read_csv('ratings.csv')
movies= pd.read_csv('movies.csv')
joined = ratings.merge(movies, on='movieId', how='left')
def film_rating(rating):
    if rating<=2:
        return('low')
    elif 2<rating<=4:
        return('averege')
    elif 4.5<=rating<=5:
        return('high')
    return()
joined['levelrating']=joined['rating'].apply(film_rating)
joined.head()

,userId,movieId,rating,timestamp,title,genres,levelrating
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,averege
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,averege
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,averege
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,high
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,high


**Задание 5**
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [40]:
import pandas as pd
geo_data = { 'Центр': ['москва', 'тула', 'ярославль'], 'Северо-Запад': ['петербург', 'псков', 'мурманск'], 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] }
data=pd.read_csv('keywords.csv')
def geo_class(keyword):
    for key in geo_data.keys():
        for city in geo_data[key]:
            if city in keyword:
                return(key)
    return('undefined')
data['region']=data['keyword'].apply(geo_class)
data.tail()

,keyword,shows,region
99995,эльдорадо старый оскол,3705,undefined
99996,frigate для firefox,3630,undefined
99997,укрсиб,3630,undefined
99998,погода в ялте на неделю,3688,undefined
99999,ггму,3630,undefined


**Задание 6**
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [101]:
import pandas as pd
ratings = pd.read_csv('ratings.csv')
movies= pd.read_csv('movies.csv')
joined = ratings.merge(movies, on='movieId', how='left')
years=[x for x in range(1950,2011)]
def production_year(title):
    for yr in years:
        if str(yr) in title:
            return(yr)
    return('1900')
joined['year']=joined['title'].apply(production_year)
joined=joined.groupby(['year','rating']).count()
joined.head()

userId  movieId  timestamp  title  genres
year rating                                           
1950 0.5          1        1          1      1       1
     1.0          3        3          3      3       3
     1.5          1        1          1      1       1
     2.0          3        3          3      3       3
     2.5          9        9          9      9       9